# Traveling salesman problem

### Simulated annealing

In [2]:
import pandas as pd
import random
import copy
import math

In [19]:
#function for drawing the initial solution
def random_shuffle(len_data):
    listaMiast = list(range(1, len_data+1))
    poczatkoweRozwiazanie = listaMiast.copy()
    random.shuffle(poczatkoweRozwiazanie)
    return poczatkoweRozwiazanie

In [20]:
#function summing up the time of performed tasks
def timeSum(data, tasksOrder):    
    columns = len(data[0])
    rows = len(data)

    #create an empty list (like in excels)
    listOfTasks=[[0]*columns for i in range(rows)]

    #complete the list with the first row
    for i in range(columns):
        listOfTasks[0][i] = data[tasksOrder[0]-1][i]+listOfTasks[0][i-1]

    #complete the list with the first column   
    for j in range(1, rows):
        listOfTasks[j][0] = data[tasksOrder[j]-1][0]+listOfTasks[j-1][0]

    #complete the rest of the list items
    for i in range(1, columns):
        for j in range(1, rows):
            listOfTasks[j][i] = data[tasksOrder[j]-1][i] + max(listOfTasks[j-1][i], listOfTasks[j][i-1])

    return listOfTasks[rows-1][columns-1]

In [21]:
#a function that finds a new solution depending on the selected neighborhood
def solution(initialSolution, newSolution, neighborhood, list):
    twoRandomList = random.sample(range(len(list)), 2)

    if (neighborhood == "SWAP"):
        newSolution[twoRandomList[0]] = initialSolution[twoRandomList[1]]
        newSolution[twoRandomList[1]] = initialSolution[twoRandomList[0]]
    elif (neighborhood == "INVERSION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution[twoRandomList[0] : twoRandomList[1] + 1] = reversed(newSolution[twoRandomList[0] : twoRandomList[1] + 1])
        else:
            newSolution[twoRandomList[1] : twoRandomList[0] + 1] = reversed(newSolution[twoRandomList[1] : twoRandomList[0] + 1])
    elif (neighborhood == "INSERTION"):
        if(twoRandomList[0] < twoRandomList[1]):
            newSolution.insert(twoRandomList[0]+1, newSolution.pop(twoRandomList[1]))
        else:
            newSolution.insert(twoRandomList[1]+1, newSolution.pop(twoRandomList[0]))
    
    return newSolution

In [22]:
def algorithm(data, alpha, t, trialsPerEpoch, tMin, neighborhood):
    list = data.values.tolist()
    initialSolution = random_shuffle(len(list))
    sum = 499999
    sumSum = 500000 #so that we can save the smallest sum that has rolled over while the algorithm is running
    sumList = []
    while t > tMin:
        trial = 0
        while trial < trialsPerEpoch:
            newSolution = copy.deepcopy(initialSolution)
            solution(initialSolution, newSolution, neighborhood, list)
            
            newSum = timeSum(list, newSolution)
            oldSum = timeSum(list, initialSolution)

            #checking whether the new solution is better than the current one
            if(newSum < oldSum):
                initialSolution = newSolution
                sum = newSum
            else:
                if(random.uniform(0, 1) < math.exp(-(newSum - oldSum)/t)):
                    initialSolution = newSolution
                    sum = newSum
            trial += 1
        t = t * alpha
        #checking if the new smallest sum is less than the "by now" smallest sum
        if(sumSum > sum):
            sumSum = sum
            sumList = initialSolution
    print("Suma", sum)
    print("Kolejność:", initialSolution)
    if(sumSum < sum):
        print("\nMożliwa niższa suma", sumSum)
        print("Kolejność:", sumList)

In [23]:
data50 = pd.read_excel("Dane_PFSP_50_20.xlsx", index_col=0)
data100 = pd.read_excel("Dane_PFSP_100_10.xlsx", index_col=0)
data200 = pd.read_excel("Dane_PFSP_200_10.xlsx", index_col=0)

In [ ]:
#example of operation
data = [data50, data100, data200]

for i in range(0,len(data)):
    print("======== Zestaw:", i+1, "========")
    n = 0
    while n < 4:
        print("POWTORZENIE:", n+1)    
        algorithm(data[i], 0.5, 500, 500, 10, "SWAP")
        n += 1